In [17]:
# This notebook takes the data as collected by oTree from the experimental sessions and organizes them in a single csv file.
# called input/originalData.csv
# Then, I take the originalData, keep only the relevant variables and add dummies identifying the treatments.
# The file with the relevant variables and treatment dummies is saved as input/rawData.csv
# Python 3.7.6
import sys
print(sys.version)

3.6.4 (v3.6.4:d48eceb, Dec 19 2017, 06:54:40) [MSC v.1900 64 bit (AMD64)]


In [18]:
import pandas as pd
import numpy as np
import shutil, os

In [19]:
# cleanFolder takes the name of a folder within the working directory and removes all files found in it.
def cleanFolder(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(e)

In [20]:
# Delete files in folders 'input' and 'temp'.
cleanFolder('input')
cleanFolder('temp')

In [21]:
# Import raw data as collected by oTree during the sessions.
# Baseline
dt1 = pd.read_csv("DATA/101817/bundlingBaseNegative (accessed 2017-10-20).csv")
dt2 = pd.read_csv("DATA/113017/bundlingBaseNegative (accessed 2017-12-01).csv")

In [22]:
# Harsh cap data
dt5 = pd.read_csv("DATA/113017/bundlingHarsh (accessed 2017-12-01).csv")

In [23]:
# Merge data from the baseline and harsh cap treatments.
original = pd.concat([dt1, dt2])
original = pd.concat([original, dt5])

c:\users\jgnun\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
c:\users\jgnun\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
# Save data as collected by oTree. These are the original data as collected by oTree.
original.to_csv("input/originalData.csv", index=False)

In [25]:
del original

In [26]:
#Take the original data.
data = pd.read_csv("input/originalData.csv")

In [27]:
# 'demo' sessions were test session I ran just to test the interface before the sessions took place. Omit from dataset.
data = data.loc[data['session.is_demo'] != 1]

In [28]:
# Keep only necessary columns
listCols = list(data.columns.values)

In [29]:
listCols

['group.id_in_subsession',
 'participant._current_app_name',
 'participant._current_page_name',
 'participant._index_in_pages',
 'participant._is_bot',
 'participant._max_page_index',
 'participant._round_number',
 'participant.code',
 'participant.exclude_from_data_analysis',
 'participant.id_in_session',
 'participant.ip_address',
 'participant.label',
 'participant.mturk_assignment_id',
 'participant.mturk_worker_id',
 'participant.payoff',
 'participant.time_started',
 'participant.visited',
 'player.boughtFour',
 'player.boughtOne',
 'player.boughtThree',
 'player.boughtTwo',
 'player.buyerGot',
 'player.buyerRent',
 'player.firstBuyerType',
 'player.id_in_group',
 'player.isEffective',
 'player.offerAny',
 'player.offerFour',
 'player.offerOne',
 'player.offerThree',
 'player.offerTwo',
 'player.packageFourPrice',
 'player.packageFourQA',
 'player.packageFourQB',
 'player.packageOnePrice',
 'player.packageOneQA',
 'player.packageOneQB',
 'player.packageThreePrice',
 'player.packa

In [30]:
# These are the relevant variables to keep.
data = data[['participant.code', 'player.setMenu', 'player.firstBuyerType',
 'player.offerAny', 'player.offerOne', 'player.offerTwo', 'player.offerThree', 'player.offerFour',
 'player.packageOneQA', 'player.packageOneQB', 'player.packageOnePrice', 'player.packageTwoQA',
 'player.packageTwoQB', 'player.packageTwoPrice', 'player.packageThreeQA', 'player.packageThreeQB',
 'player.packageThreePrice', 'player.packageFourQA', 'player.packageFourQB', 'player.packageFourPrice',
 'player.reject', 'player.boughtOne', 'player.boughtTwo', 'player.boughtThree', 'player.boughtFour',
 'player.profitOne', 'player.profitTwo', 'player.profitThree', 'player.profitFour', 'player.buyerRent',
 'player.buyerGot', 'player.pseudoPayoff', 'subsession.round_number',
 'session.code']]

In [31]:
# Renaming columns to something practical.
data = data.rename(columns={'participant.code': 'id', 'session.code': 'sessionCode',
                            'subsession.round_number': 'period',
                            'player.firstBuyerType': 'buyerType'})

In [32]:
# Session codes for baseline (base) and cap (harsh) treatment.
base = ['gtyxuold', 'o02inqaf', 's100cw5b']
harsh = ['l6mxjzgf', 'zpnfshc7', 'ofdj2tpz']
allS = base + harsh
data = data[data['sessionCode'].isin(allS)]

In [33]:
# Rename columns indicating alternative packages. to simplify coding.
data.columns = data.columns.str.replace("player.", "")
data.columns = data.columns.str.replace("packageOne", "one")
data.columns = data.columns.str.replace("packageTwo", "two")
data.columns = data.columns.str.replace("packageThree", "three")
data.columns = data.columns.str.replace("packageFour", "four")

In [34]:
# In one session, not all computers were used, so there are some empty cells.
data = data[np.isfinite(data['buyerType'])]

In [35]:
# Generate dummy variables indicating treatments.
treatmentsDF = pd.get_dummies(data['sessionCode'])

In [36]:
# Treatment dummies
treatmentsDF['Base'] = treatmentsDF['gtyxuold'] + treatmentsDF['o02inqaf'] + treatmentsDF['s100cw5b']
treatmentsDF['Harsh'] = treatmentsDF['l6mxjzgf'] + treatmentsDF['zpnfshc7'] + treatmentsDF['ofdj2tpz']

In [37]:
data['Base'] = treatmentsDF['Base']
data['Harsh'] = treatmentsDF['Harsh']

In [38]:
data = data.rename(columns={'pseudoPayoff': 'periodPayoff'})

In [39]:
data.head()

id       setMenu  buyerType  offerAny  offerOne  offerTwo  \
11  1nr668lr   One package        2.0         1         1         0   
12  vbzunngw  Two packages        2.0         1         0         1   
13  ccxbxg4p  Two packages        3.0         1         0         1   
14  dqlq3d7v   One package        3.0         1         1         0   
15  d6jotgs5  Two packages        3.0         1         0         1   

    offerThree  offerFour  oneQA  oneQB  ...  profitTwo  profitThree  \
11           0          0     90     90  ...          0            0   
12           0          0     40     30  ...          0            0   
13           0          0    200    200  ...          0            0   
14           0          0     15     20  ...          0            0   
15           0          0     10     10  ...         49            0   

    profitFour  buyerRent  buyerGot  periodPayoff  period  sessionCode  Base  \
11           0  47.170825         1        157.60       1     gtyxuold     1   
12           0  54.640586         1         90.00       1     gtyxuold     1   
13           0   0.553391         1        193.00       1     gtyxuold     1   
14           0  15.811873         1         88.75       1     gtyxuold     1   
15           0   5.901699         2         49.90       1     gtyxuold     1   

    Harsh  
11      0  
12      0  
13      0  
14      0  
15      0  

[5 rows x 36 columns]

In [40]:
#Save the data for analysis.
data.to_csv("input/rawData.csv", index=False)